In [1]:
#!fsharp
// installation of paket integration (did not find out to do it properly)
let installDesignTime () = 
    let home = 
        match System.Environment.OSVersion.Platform with
        | PlatformID.Unix | PlatformID.MacOSX -> Environment.GetEnvironmentVariable("HOME")
        | _ -> Environment.ExpandEnvironmentVariables("%HOMEDRIVE%%HOMEPATH%")
    let target = System.IO.Path.Combine(home,@".nuget\packages\microsoft.dotnet-interactive\1.0.156101\tools\netcoreapp3.1\any\FSharp.DependencyManager.Paket.dll")
    System.IO.File.Copy(@"packages\build\FSharp.DependencyManager.Paket\lib\netstandard2.0\FSharp.DependencyManager.Paket.dll", target)

In [1]:
#!fsharp
installDesignTime()

In [1]:
#!fsharp
// add a lot packages...
#r "paket: nuget FSharp.Data.Adaptive"
#r "paket: nuget Aardvark.Base"
#r "paket: nuget Aardvark.Base.Incremental"
#r "paket: nuget Aardvark.Base.FSharp"
#r "paket: nuget Aardvark.Base.Rendering"
#r "paket: nuget Aardvark.Application.Slim"
#r "paket: nuget Aardvark.Application.Slim.GL"
#r "paket: nuget Aardvark.Application.Slim.Vulkan"
#r "paket: nuget Aardvark.SceneGraph"
#r "paket: nuget Aardvark.SceneGraph.IO"
#r "paket: nuget Aardvark.Service"
#r "paket: nuget Aardvark.UI"
#r "paket: nuget Aardvark.UI.Primitives"

ok paket


err paket


In [1]:
#!fsharp
// can be removed using project reference in the future
#r @"bin\Debug\netcoreapp3.1\NotebookApp.dll"

In [1]:
#!fsharp
open Aardvark.Notebooks
Notebooks.init()

In [1]:
#!fsharp
let show (v : Aardvark.Notebooks.NotebookUtilities.NotebookView) = 
    let html = sprintf "<iframe src='%s' width='400' height='200'></frame>" v.Url
    Html.ToHtmlContent html

In [1]:
#!fsharp
open NotebookApp

let app = new Aardvark.Application.Slim.VulkanApplication()

In [1]:
#!fsharp
open Aardvark.Base
open Aardvark.UI
open Aardvark.UI.Primitives
open FSharp.Data.Adaptive
open NotebookApp.Model
open Aardvark.Base.Rendering

In [1]:
#!fsharp
let napp = NotebookApp(app.Runtime, App.app)
napp.AddView(App.view) |> show


In [1]:
#!fsharp
napp.AddView(App.view) |> show

In [1]:
#!fsharp
let viewAnnotations (m : AdaptiveModel) =
    body [style "background-color: #272727"] [
        require Html.semui (
            Incremental.div (AttributeMap.ofList [style "position: fixed; left: 20px; top: 20px"]) <|
                alist {
                    for p in m.annotations do
                        yield div [style "color:black"] [text (sprintf "(%.2f,%.2f,%.2f)" p.X p.Y p.Z)]
            
            }
        )
    ]
napp.AddView(viewAnnotations) |> show


Stopped due to error


Error: input.fsx (1,26)-(1,39) typecheck error The type 'AdaptiveModel' is not defined.
input.fsx (2,5)-(2,9) typecheck error The value or constructor 'body' is not defined. Maybe you want one of the following:
   box
input.fsx (12,1)-(12,5) typecheck error The value, namespace, type or module 'napp' is not defined. Maybe you want one of the following:
   nan
input.fsx (12,34)-(12,38) typecheck error The value or constructor 'show' is not defined.

In [1]:
#!fsharp
let points = napp.State.annotations |> AList.force 
points |> Seq.map (fun p -> p * 2.0)

index Elements MajorDim MinorDim AnyNaN AllNaN AnyInfinity AllInfinity AnyPositiveInfinity AllPositiveInfinity AnyNegativeInfinity AllNegativeInfinity AnyTiny AllTiny IsNaN IsInfinity IsFinite LengthSquared Length Norm1 Norm2 .. 0 [ 1.685069594902037, 0.5449862265577234, 0.8697285417102485 ] 0 1 False False False False False False False False False False False False True 3.8928972630663763 1.9730426409650594 3.099784363170009 1.9730426409650594 1 [ -0.01997682068572626, 1.5786815573319117, 1.1690903157868329 ] 1 0 False False False False False False False False False False False False True 3.8594066992911764 1.9645372735815365 2.767748693804471 1.9645372735815365 2 [ 1.0353300055451546, 1.3005796013403788, 1.0809193893920672 ] 1 0 False False False False False False False False False False False False True 3.931802246168548 1.982877264524597 3.4168289962776006 1.982877264524597 3 [ 0.7430940707841782, 1.7879125672677292, 0.3735101825770277 ] 1 2 False False False False False False False False False False False False True 3.888330002717208 1.9718848857672215 2.904516820628935 1.9718848857672215 4 [ 0.829722221634581, 0.8690093229390015, 1.5785264576811626 ] 2 0 False False False False False False False False False False False False True 3.935361946028566 1.9837746711833393 3.277258002254745 1.9837746711833393 5 [ 0.589126428828469, 1.1192018849116065, 1.5030975395596857 ] 2 0 False False False False False False False False False False False False True 3.858985021764259 1.9644299482965177 3.2114258532997613 1.9644299482965177

In [1]:
#!fsharp
